In [2]:
import pandas as pd
import numpy as np
import pdfplumber
import re
from src.config import BLD, RAW

# 读取基本信息数据
def load_basic_info():
    df_basic_file = RAW / 'csmar' / '基本信息' / '上市公司基本信息年度表215937759' / 'STK_LISTEDCOINFOANL.csv'
    df_basic = pd.read_csv(df_basic_file, dtype={'Symbol': str}).fillna(0)
    df_basic["year"] = pd.to_datetime(df_basic["EndDate"]).dt.year.astype(int)
    df_basic.rename(columns={'Symbol': 'Stkcd'}, inplace=True)
    df_basic = df_basic[(df_basic['Stkcd'] >= "000001") & (df_basic['Stkcd'] <= "679999")]
    df_basic['Industry_Main'] = df_basic['IndustryCode'].str[0]
    return df_basic

def filter_companies_with_stable_industry(df_basic):
    """
    过滤掉行业变动的公司。
    :param df_basic: 包含行业信息的基础公司数据 DataFrame
    :return: 过滤后的公司代码列表
    """
    industry_changes = df_basic[df_basic['year'].between(2014, 2019)].groupby('Stkcd')['Industry_Main'].apply(set)
    companies_with_changes = industry_changes[industry_changes.apply(len) > 1].index
    return companies_with_changes




def load_subsidiary_data(raw_path):
    """
    加载上市公司子公司数据。
    :param raw_path: 数据根目录路径
    :return: DataFrame
    """
    df_subsidiary_file = raw_path / 'csmar' / '基本信息' / '上市公司子公司情况表220415924' / 'FN_Fn061.csv'
    df_subsidiary = pd.read_csv(df_subsidiary_file, dtype={'Stkcd': str})
    df_subsidiary["year"] = pd.to_datetime(df_subsidiary["EndDate"]).dt.year.astype(int)
    df_subsidiary = df_subsidiary[(df_subsidiary['Stkcd'] >= "000001") & (df_subsidiary['Stkcd'] <= "679999")]
    return df_subsidiary


def merge_company_data(df_basic, df_subsidiary, companies_with_changes):
    df_basic = df_basic.loc[~df_basic['Stkcd'].isin(companies_with_changes), ['Stkcd', 'year', 'FullName']]
    df_subsidiary = df_subsidiary.loc[~df_subsidiary['Stkcd'].isin(companies_with_changes), ['Stkcd', 'year', 'FN_Fn06101']]

    df_basic.rename(columns={'FullName': 'company'}, inplace=True)
    df_subsidiary.rename(columns={'FN_Fn06101': 'company'}, inplace=True)

    return pd.concat([df_basic, df_subsidiary], ignore_index=True)


###############################
def load_export_data(raw_path):
    """
    加载出口数据。
    :param raw_path: 数据根目录路径
    :return: DataFrame
    """
    df_export_file = raw_path / 'cn_custom_data' / '2014_2016_export_data.parquet'
    return pd.read_parquet(df_export_file)

def merge_export_data(merged_df_list, df_export):
    """
    合并母公司和子公司数据与出口数据。
    :param merged_df_list: 包含母公司和子公司名称的 DataFrame
    :param df_export: 出口数据 DataFrame
    :return: 合并后的 DataFrame
    """
    result_df = pd.merge(
        merged_df_list,
        df_export[['Company_Name', 'Year', 'Country_Name', 'Export_Amount', 'Product_Code']],
        left_on=['company', 'year'],
        right_on=['Company_Name', 'Year'],
        how='inner'
    )
    result_df['hs_code'] = result_df['Product_Code'].astype(str).str[:6].astype(int)
    return result_df

def filter_usa_exports(result_df):
    """
    筛选出出口美国的数据，并转换金额。
    :param result_df: 合并后的 DataFrame
    :return: 处理后的 DataFrame
    """
    exchange_rates = {2014: 6.128333, 2015: 6.205000, 2016: 6.614167}
    return result_df.loc[result_df['Country_Name'] == '美国'].assign(
        export=lambda df: df['Export_Amount'] * df['year'].map(exchange_rates)
    )

def aggregate_export_data(df_usa):
    """
    筛选出至少有 2 年数据的公司。
    :param df_usa: 出口美国的数据 DataFrame
    :return: 汇总后的 DataFrame
    """
    grouped_df = df_usa.groupby(['Stkcd', 'year', 'hs_code'], as_index=False).agg({
        'Export_Amount': 'sum',
        'export': 'sum'
    })
    grouped_df.sort_values(by=['Stkcd', 'year'], inplace=True)

    company_year_counts = grouped_df.groupby('Stkcd')['year'].nunique()
    valid_companies = company_year_counts[company_year_counts >= 2].index
    return grouped_df[grouped_df['Stkcd'].isin(valid_companies)]

def compute_avg_export(df_usa):
    """
    计算 2014-2016 年公司-商品组合的平均出口金额。
    :param df_usa: 处理后的出口数据 DataFrame
    :return: 计算平均出口金额后的 DataFrame
    """
    df_usa_avg = df_usa.groupby(['Stkcd', 'hs_code'])['export'].mean().reset_index()
    df_usa_avg.rename(columns={'export': 'avg_export_amount'}, inplace=True)
    return df_usa_avg

##################
def extract_hts_codes_from_pdf(pdf_path, page_range=None):
    """
    从 PDF 提取 HTS 代码，并格式化为 6 位代码。
    
    :param pdf_path: PDF 文件路径
    :param page_range: 需要提取的页码范围
    :return: 排序后的 6 位 HTS 代码列表
    """
    all_text = ""
    with pdfplumber.open(pdf_path) as pdf:
        if page_range:
            for page_number in page_range:
                page = pdf.pages[page_number]
                all_text += page.extract_text() + " "
        else:
            for page in pdf.pages:
                all_text += page.extract_text() + " "

    # 提取符合 8 位格式的 HTS 代码
    hts_codes = re.findall(r'\b\d{4}\.\d{2}\.\d{2}\b', all_text)

    # 格式化为 6 位代码（去掉圆点和后两位）
    hts_prefixes = {code.replace(".", "")[:6] for code in hts_codes}

    return sorted(map(int, hts_prefixes))  # 返回排序后的整数列表

def process_hts_codes(raw_path):
    """
    处理多个 PDF 文件中的 HTS 代码，并进行集合运算。
    
    :param raw_path: 数据根目录路径
    :return: hts_list_1_3, hts_list_4
    """
    hts_list_1 = extract_hts_codes_from_pdf(raw_path / 'us_tariff' / 'FRN301.pdf')
    hts_list_2 = extract_hts_codes_from_pdf(raw_path / 'us_tariff' / 'list_2.pdf', page_range=range(4, 9))
    hts_list_3 = extract_hts_codes_from_pdf(raw_path / 'us_tariff' / 'list_3.pdf', page_range=range(3, 28))
    hts_list_4 = extract_hts_codes_from_pdf(raw_path / 'us_tariff' / 'list_4.pdf', page_range=range(3, 25))

    # 计算集合运算
    set1 = set(hts_list_1)
    set2 = set(hts_list_2)
    set3 = set(hts_list_3)
    set4 = set(hts_list_4)

    set5 = set1 | set2 | set3  # 1-3号文件的HTS代码集合
    set6 = set4 - set5         # 4号文件中不在 1-3 号文件中的 HTS 代码

    hts_list_1_3 = sorted(set5)
    hts_list_4 = sorted(set6)

    return hts_list_1_3, hts_list_4
#####################################
def mark_affected_products(df_usa_avg, hts_list_1_3, hts_list_4):
    """
    标记受影响的 HS 代码。
    
    :param df_usa_avg: 包含公司出口数据的 DataFrame
    :param hts_list_1_3: 受 1-3 号清单影响的 HS 代码列表
    :param hts_list_4: 受 4 号清单影响的 HS 代码列表
    :return: 更新后的 DataFrame
    """
    df_usa_avg['affected_1_3'] = df_usa_avg['hs_code'].isin(hts_list_1_3)
    df_usa_avg['affected_4'] = df_usa_avg['hs_code'].isin(hts_list_4)
    return df_usa_avg

def calculate_export_impacts(df_usa_avg):
    """
    计算公司总出口金额及受影响金额。

    :param df_usa_avg: 包含出口数据和受影响标记的 DataFrame
    :return: 计算后的 DataFrame
    """
    # 按公司代码计算总金额
    df_total = df_usa_avg.groupby('Stkcd')['avg_export_amount'].sum().reset_index()
    df_total.rename(columns={'avg_export_amount': 'total_export_amount'}, inplace=True)

    # 计算受 1-3 号清单影响的金额
    df_1_3 = df_usa_avg[df_usa_avg['affected_1_3']].groupby('Stkcd')['avg_export_amount'].sum().reset_index()
    df_1_3.rename(columns={'avg_export_amount': 'amount_affected_1_3'}, inplace=True)

    # 计算受 4 号清单影响的金额
    df_4 = df_usa_avg[df_usa_avg['affected_4']].groupby('Stkcd')['avg_export_amount'].sum().reset_index()
    df_4.rename(columns={'avg_export_amount': 'amount_affected_4'}, inplace=True)

    # 合并结果
    result = pd.merge(df_total, df_1_3, on='Stkcd', how='left')
    result = pd.merge(result, df_4, on='Stkcd', how='left')

    # 填充 NaN 值
    result.fillna(0, inplace=True)
    
    return result

###############################
def load_income_data(raw_path):
    """
    加载公司利润表数据，并进行基础清理。

    :param raw_path: 数据根目录路径（Path 对象）
    :return: 处理后的 DataFrame
    """
    income_columns = {"Stkcd": "Stkcd", "Accper": "year", "B001100000": "revenue"}
    df_income_file = raw_path / 'csmar' / '基本信息' / '利润表000222262' / 'FS_Comins.csv'
    
    df_income = pd.read_csv(df_income_file, usecols=income_columns.keys(), dtype={'Stkcd': str}).rename(columns=income_columns)
    df_income["year"] = pd.to_datetime(df_income["year"]).dt.year
    df_income = df_income[(df_income['Stkcd'] >= "000001") & (df_income['Stkcd'] <= "679999")]
    
    return df_income

def merge_income_with_industry(df_income, df_basic):
    """
    将利润数据与行业数据合并，并去除 NaN。

    :param df_income: 公司利润数据 DataFrame
    :param df_basic: 包含行业信息的基础公司数据 DataFrame
    :return: 合并后的 DataFrame
    """
    merged_df = pd.merge(df_income, df_basic[['Stkcd', 'year', 'Industry_Main']], on=['Stkcd', 'year'], how='left')
    merged_df = merged_df.dropna()  # 去掉包含 NaN 的行
    return merged_df

def compute_average_revenue(merged_df):
    """
    计算 2014-2016 年的公司平均收入。

    :param merged_df: 包含利润数据的 DataFrame
    :return: 计算后的平均收入 DataFrame
    """
    average_revenue_df = (
        merged_df[(merged_df['year'] >= 2014) & (merged_df['year'] <= 2016)]
        .groupby(['Stkcd', 'Industry_Main'], as_index=False)['revenue']
        .mean()
        .reset_index()
        .rename(columns={'revenue': 'average_revenue'})
    )
    return average_revenue_df

def merge_with_export_data(average_revenue_df, result):
    """
    合并平均收入数据与出口影响数据，并计算受影响比例。

    :param average_revenue_df: 计算后的平均收入 DataFrame
    :param result: 计算出口影响的 DataFrame
    :return: 合并后的 DataFrame
    """
    merged_df = pd.merge(average_revenue_df, result, on='Stkcd', how='left').fillna(0)
    merged_df['ratio_affected_1_3'] = merged_df['amount_affected_1_3'] / merged_df['average_revenue']
    merged_df['ratio_affected_4'] = merged_df['amount_affected_4'] / merged_df['average_revenue']
    merged_df = merged_df[merged_df["total_export_amount"] > 0]
    
    return merged_df

#############################
def classify_impact(row, threshold_1_3=0.1, threshold_4=0.1):
    """
    根据受影响比例分类公司影响类别。
    
    :param row: DataFrame 的行
    :param threshold_1_3: 受影响 1-3 清单的阈值
    :param threshold_4: 受影响 4 清单的阈值
    :return: 影响类别（1, 2, 3, 4）
    """
    if row['ratio_affected_1_3'] > threshold_1_3 and row['ratio_affected_4'] > threshold_4:
        return 3  # 受 1-3 号和 4 号清单影响
    elif row['ratio_affected_1_3'] > threshold_1_3:
        return 1  # 受 1-3 号清单影响
    elif row['ratio_affected_4'] > threshold_4:
        return 2  # 受 4 号清单影响
    else:
        return 4  # 未受影响

def classify_and_filter_industry_c(merged_df):
    """
    计算影响类别，并筛选出行业 'C' 的公司。

    :param merged_df: 计算受影响比例后的 DataFrame
    :return: 筛选出的 DataFrame
    """
    # 新增影响分类列
    merged_df['impact_category'] = merged_df.apply(classify_impact, axis=1)

    # 筛选行业 'C' 的公司
    df_c = merged_df.loc[merged_df['Industry_Main'] == 'C', ['Stkcd', 'impact_category']]

    # 标记 treatment 变量
    df_c = df_c.assign(
        treatment=np.where(df_c['impact_category'].isin({1, 2, 3}), 1, 0),
        year='2018Q3'
    )
    return df_c[['Stkcd', 'treatment', 'year']]



In [6]:
from pathlib import Path

BASE = Path().resolve().parent
BASE


PosixPath('/Users/hengdegao/python/thesis')

In [7]:


raw_path = BASE / "data" / "original_data"

df_basic = load_basic_info()

    # 加载子公司数据
df_subsidiary = load_subsidiary_data(raw_path)

    # 过滤行业变动公司
companies_with_changes = filter_companies_with_stable_industry(df_basic)

    # 合并母公司和子公司数据
merged_df_list = merge_company_data(df_basic, df_subsidiary, companies_with_changes)

df_export = load_export_data(raw_path)

result_df = merge_export_data(merged_df_list, df_export)

In [8]:
df_usa = filter_usa_exports(result_df)

    # 计算平均出口金额
df_usa_avg = compute_avg_export(df_usa)
df_usa_avg

,Stkcd,hs_code,avg_export_amount
0,000009,250410,2.030051e+05
1,000009,284290,2.976375e+03
2,000009,380110,2.327422e+07
3,000009,381590,3.442277e+05
4,000009,392350,1.448503e+04
...,...,...,...
26140,603997,392690,1.804181e+04
26141,603997,600632,1.218006e+05
26142,603997,731814,2.830625e+01
26143,603997,940190,3.895868e+06


In [9]:
hts_list_1_3, hts_list_4 = process_hts_codes(raw_path)
hts_list_1_3

[20329,
 20610,
 20810,
 20890,
 21019,
 30111,
 30119,
 30191,
 30192,
 30193,
 30194,
 30195,
 30199,
 30211,
 30213,
 30214,
 30219,
 30221,
 30222,
 30223,
 30224,
 30229,
 30231,
 30232,
 30233,
 30234,
 30235,
 30236,
 30239,
 30241,
 30242,
 30243,
 30244,
 30245,
 30246,
 30247,
 30249,
 30251,
 30252,
 30253,
 30254,
 30255,
 30256,
 30259,
 30271,
 30272,
 30273,
 30274,
 30279,
 30281,
 30282,
 30283,
 30284,
 30285,
 30289,
 30291,
 30292,
 30299,
 30311,
 30312,
 30313,
 30314,
 30319,
 30323,
 30324,
 30325,
 30326,
 30329,
 30331,
 30332,
 30333,
 30334,
 30339,
 30341,
 30342,
 30343,
 30344,
 30345,
 30346,
 30349,
 30351,
 30353,
 30354,
 30355,
 30356,
 30357,
 30359,
 30363,
 30364,
 30365,
 30366,
 30367,
 30368,
 30381,
 30382,
 30383,
 30384,
 30389,
 30391,
 30392,
 30399,
 30431,
 30432,
 30433,
 30439,
 30441,
 30442,
 30443,
 30444,
 30445,
 30446,
 30447,
 30448,
 30449,
 30451,
 30452,
 30453,
 30454,
 30455,
 30456,
 30457,
 30459,
 30461,
 30462,
 30463,


In [10]:
df_usa_avg = mark_affected_products(df_usa_avg, hts_list_1_3, hts_list_4)
df_usa_avg

,Stkcd,hs_code,avg_export_amount,affected_1_3,affected_4
0,000009,250410,2.030051e+05,False,False
1,000009,284290,2.976375e+03,True,False
2,000009,380110,2.327422e+07,True,False
3,000009,381590,3.442277e+05,True,False
4,000009,392350,1.448503e+04,True,False
...,...,...,...,...,...
26140,603997,392690,1.804181e+04,True,False
26141,603997,600632,1.218006e+05,True,False
26142,603997,731814,2.830625e+01,True,False
26143,603997,940190,3.895868e+06,True,False


In [11]:
export_impact_df = calculate_export_impacts(df_usa_avg)
export_impact_df

,Stkcd,total_export_amount,amount_affected_1_3,amount_affected_4
0,000009,3.096630e+07,3.076129e+07,0.000000e+00
1,000011,1.719292e+06,2.390050e+05,0.000000e+00
2,000012,3.265556e+06,3.265556e+06,0.000000e+00
3,000016,2.443334e+08,1.721093e+08,5.897270e+05
4,000020,5.433077e+06,9.845915e+05,0.000000e+00
...,...,...,...,...
1369,603987,2.776398e+07,1.254992e+06,0.000000e+00
1370,603988,7.608541e+05,7.608541e+05,0.000000e+00
1371,603989,6.877404e+06,6.877404e+06,0.000000e+00
1372,603996,1.953793e+08,1.811133e+08,2.419334e+06


In [12]:
df_income = load_income_data(raw_path)

    # 合并行业信息
merged_income_df = merge_income_with_industry(df_income, df_basic)

    # 计算公司平均收入
avg_revenue_df = compute_average_revenue(merged_income_df)


In [13]:
final_df = merge_with_export_data(avg_revenue_df, export_impact_df)
final_df

,index,Stkcd,Industry_Main,average_revenue,total_export_amount,amount_affected_1_3,amount_affected_4,ratio_affected_1_3,ratio_affected_4
6,6,000009,S,3.250126e+09,3.096630e+07,3.076129e+07,0.000000e+00,0.009465,0.000000
9,9,000011,K,8.057665e+08,1.719292e+06,2.390050e+05,0.000000e+00,0.000297,0.000000
10,10,000012,C,5.232819e+09,3.265556e+06,3.265556e+06,0.000000e+00,0.000624,0.000000
12,12,000016,C,1.310109e+10,2.443334e+08,1.721093e+08,5.897270e+05,0.013137,0.000045
17,17,000020,C,4.265669e+08,5.433077e+06,9.845915e+05,0.000000e+00,0.002308,0.000000
...,...,...,...,...,...,...,...,...,...
2964,2964,603987,C,1.096748e+09,2.776398e+07,1.254992e+06,0.000000e+00,0.001144,0.000000
2965,2965,603988,C,2.132795e+08,7.608541e+05,7.608541e+05,0.000000e+00,0.003567,0.000000
2966,2966,603989,C,9.966938e+08,6.877404e+06,6.877404e+06,0.000000e+00,0.006900,0.000000
2969,2969,603996,C,2.650825e+09,1.953793e+08,1.811133e+08,2.419334e+06,0.068323,0.000913


In [14]:
final_df = final_df[final_df['Industry_Main']=='C'].copy()
final_df

,index,Stkcd,Industry_Main,average_revenue,total_export_amount,amount_affected_1_3,amount_affected_4,ratio_affected_1_3,ratio_affected_4
10,10,000012,C,5.232819e+09,3.265556e+06,3.265556e+06,0.000000e+00,0.000624,0.000000
12,12,000016,C,1.310109e+10,2.443334e+08,1.721093e+08,5.897270e+05,0.013137,0.000045
17,17,000020,C,4.265669e+08,5.433077e+06,9.845915e+05,0.000000e+00,0.002308,0.000000
18,18,000021,C,1.073711e+10,1.252126e+08,4.237741e+07,8.092373e+07,0.003947,0.007537
26,26,000030,C,3.484994e+09,1.024969e+07,1.024969e+07,0.000000e+00,0.002941,0.000000
...,...,...,...,...,...,...,...,...,...
2964,2964,603987,C,1.096748e+09,2.776398e+07,1.254992e+06,0.000000e+00,0.001144,0.000000
2965,2965,603988,C,2.132795e+08,7.608541e+05,7.608541e+05,0.000000e+00,0.003567,0.000000
2966,2966,603989,C,9.966938e+08,6.877404e+06,6.877404e+06,0.000000e+00,0.006900,0.000000
2969,2969,603996,C,2.650825e+09,1.953793e+08,1.811133e+08,2.419334e+06,0.068323,0.000913


In [15]:
threshold_1_3 = 0.1
threshold_4 = 0.1

def classify_impact(row):
    if row['ratio_affected_1_3'] > threshold_1_3 and row['ratio_affected_4'] > threshold_4:
        return 3  # both impacted
    elif row['ratio_affected_1_3'] > threshold_1_3:
        return 1  # list1-3
    elif row['ratio_affected_4'] > threshold_4:
        return 2  # list4
    else:
        return 4  # unaffected

# 新增分类列
final_df['impact_category'] = final_df.apply(classify_impact, axis=1)


In [16]:
final_df

,index,Stkcd,Industry_Main,average_revenue,total_export_amount,amount_affected_1_3,amount_affected_4,ratio_affected_1_3,ratio_affected_4,impact_category
10,10,000012,C,5.232819e+09,3.265556e+06,3.265556e+06,0.000000e+00,0.000624,0.000000,4
12,12,000016,C,1.310109e+10,2.443334e+08,1.721093e+08,5.897270e+05,0.013137,0.000045,4
17,17,000020,C,4.265669e+08,5.433077e+06,9.845915e+05,0.000000e+00,0.002308,0.000000,4
18,18,000021,C,1.073711e+10,1.252126e+08,4.237741e+07,8.092373e+07,0.003947,0.007537,4
26,26,000030,C,3.484994e+09,1.024969e+07,1.024969e+07,0.000000e+00,0.002941,0.000000,4
...,...,...,...,...,...,...,...,...,...,...
2964,2964,603987,C,1.096748e+09,2.776398e+07,1.254992e+06,0.000000e+00,0.001144,0.000000,4
2965,2965,603988,C,2.132795e+08,7.608541e+05,7.608541e+05,0.000000e+00,0.003567,0.000000,4
2966,2966,603989,C,9.966938e+08,6.877404e+06,6.877404e+06,0.000000e+00,0.006900,0.000000,4
2969,2969,603996,C,2.650825e+09,1.953793e+08,1.811133e+08,2.419334e+06,0.068323,0.000913,4


In [17]:
# 生成交叉表
industry_impact_stats = pd.crosstab(final_df['Industry_Main'], final_df['impact_category'])

# 重命名 impact_category 的列名
industry_impact_stats = industry_impact_stats.rename(columns={1: 'list1-3', 2: 'list4', 3: 'both', 4: 'unaffected'})

industry_impact_stats

impact_category,list1-3,list4,unaffected
Industry_Main,,,
C,83,15,1060


In [18]:
affected_stkcd_list = final_df[final_df['impact_category'].isin([1, 2, 3])][['Stkcd', 'impact_category']]

affected_stkcd_list

,Stkcd,impact_category
135,000541,1
443,002004,1
447,002009,1
521,002083,2
522,002084,1
...,...,...
2871,603600,1
2903,603703,1
2920,603800,1
2957,603958,2


In [19]:
output_path = BLD / 'affected_companies.parquet'
affected_stkcd_list.to_parquet(output_path, index=False)


In [20]:
aff = pd.read_parquet(output_path)

In [21]:
aff

,Stkcd,impact_category
0,000541,1
1,002004,1
2,002009,1
3,002083,2
4,002084,1
...,...,...
93,603600,1
94,603703,1
95,603800,1
96,603958,2
